## Documentación uso de Cartera

### Los import necesarios

In [1]:
import pyodbc

from Activo import *
from Accion import *
from Bono import *
from Derivado import *
from DerivadosTipos.DerivadosSCC import *

from Cartera import *

### Definimos la conexión a base de datos

In [2]:
server = '172.16.1.38'
username = 'sa'
password = 'qwerty123'
driver = '{ODBC Driver 17 for SQL Server}'
cn = pyodbc.connect('DRIVER=' + driver + ';SERVER=' + server + ';UID=' + username + ';PWD=' + password)

### Primero, definimos los inputs (activos) que se le entregarán a la cartera

#### Construimos una acción
 A partir de un archivo xlsx se extrae la columna 'Adj Close' para definir el historico de la acción. Hay que insertar la ruta correcta del archivo.

In [3]:
#archivo = pd.read_excel('C:\\Users\\groso\\Desktop\\Practica\\Intento\\Cartera_definitiva\\AMZN.xlsx')    
archivo = pd.read_excel('C:\\Users\\Lenovo\\Documents\\Universidad\\Practica\\Cartera_V2\\Practica\\Intento\\Cartera\\AMZN.xlsx')
columnas = ["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume"]
archivo = archivo[columnas][:60]
accion = pd.DataFrame()
accion['Moneda'] = ['USD']
accion['Historico'] = [[archivo["Adj Close"]]]
accion['Nombre'] = "Amazon"
accion['Inversion'] = [500000]

#### Definimos un bono

In [4]:
bono = pd.DataFrame()
bono['Riesgo'] = ['AAA']  
bono['Moneda'] = ["CLP"]
bono['TablaDesarrollo'] = ["1#25-09-2018#2,2252#0#100#2,2252|2#25-03-2019#2,2252#0#100#2,2252|3#25-09-2019#2,2252#0#100#2,2252|4#25-03-2020#2,2252#0#100#2,2252|5#25-09-2020#2,2252#0#100#2,2252|6#25-03-2021#2,2252#100#0#102,2252"]
bono['Convencion'] = ["ACT360"]
bono['FechaEmision'] = ['2018-02-20']

#### Declaramos un derivado

In [5]:
info_derivado = dict()
info_derivado["Tipo"] = 'SCC'
info_derivado["ID_Key"] = ""
info_derivado["Administradora"] = "Admin"
info_derivado["Fondo"] = "Fondo"
info_derivado["Contraparte"] = "Contraparte"
info_derivado["ID"] = 3
info_derivado["Nemotecnico"] = ""
info_derivado["Mercado"] = "Local"
fecha = datetime.date(2019, 12, 20)
hora = '1700'
info_derivado["FechaEfectiva"] = '12/11/2019'
info_derivado["FechaVenc"] = '12/11/2021'
info_derivado["AjusteFeriados"] = "CL"
info_derivado["NocionalActivo"] = 10*(10**9)
info_derivado["MonedaActivo"] = 'CLP'
info_derivado["MonedaBase"] = 'CLP'
info_derivado["TipoTasaActivo"] = 'Fija'
info_derivado["TipoTasaPasivo"] = 'Flotante'
info_derivado["TasaActivo"] = 1.45
info_derivado["TasaPasivo"] = -1000
info_derivado["FrecuenciaActivo"] = "Semi annual"
info_derivado["FrecuenciaPasivo"] = info_derivado["FrecuenciaActivo"]

info1 = pd.DataFrame([info_derivado])
derivado_info = DerivadosSCC(fecha, hora, info1, cn)
derivado = pd.DataFrame()
derivado['Derivado'] = [derivado_info]

C:\Users\Lenovo\Documents\Universidad\Practica\Cartera_V2\Practica\Intento\Cartera\DerivadosTipos\DerivadosSCC.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  info_cartera.MonedaActivo[0] = "CLP"


### La cartera

Ahora entregamos estos activos a la cartera para que esta los pueda procesar. A Cartera se le debe entregar un dataFrame con las acciones, otro para bonos y aparte para derivados, luego se declara la moneda con la que se desea evaluar los indicadores de la cartera, la fecha a valorizar y la conexión a base de datos.

In [6]:
cartera = Cartera(accion, bono, derivado, 'CLP', datetime.date(2019,2,1), cn)

{'AAA':             0
0    0.652322
1    1.500491
2    0.401804
3    2.599703
4   11.095605
5   44.588459
6    0.000000
7    0.000000
8    0.000000
9    0.000000
10   0.000000
11   0.000000
12   0.000000
13   0.000000}
[]
[500000.0]
[500000.0, 0.6523217430760384, 1.5004906717530693, 0.40180404193453967, 2.599703440484092, 11.095605038846188, 44.58845927820783, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[500000.0, 0.6523217430760384, 1.5004906717530693, 0.40180404193453967, 2.599703440484092, 11.095605038846188, 44.58845927820783, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 94713829.62426706, 15192455599.267342, 3483321135.2953367, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


Para inicializar los cálculos, se debe pedir que se seteen los historicos, retornos y volatilidades respectivas.

In [7]:
cartera.set_hist_ret_vol_totales()

#### Acciones

In [8]:
acciones = cartera.get_acciones()
acciones

In [9]:
accion = acciones[0]
accion.get_retornos()

,Amazon
0,0.000000
1,0.005883
2,-0.018282
3,-0.017160
4,0.026717
5,-0.024335
6,-0.022977
7,0.013726
8,-0.013726
9,-0.001650


In [10]:
accion.get_volatilidad()

,0
Amazon,0.017711


#### Bonos


In [11]:
bonos = cartera.get_bonos()
bonos

In [12]:
bono = bonos[0]
bono.get_historicos()

,CLP#30#AAA,CLP#90#AAA,CLP#180#AAA,CLP#360#AAA,CLP#720#AAA,CLP#1080#AAA,CLP#1440#AAA,CLP#1800#AAA,CLP#2520#AAA,CLP#3240#AAA,CLP#3600#AAA,CLP#5400#AAA,CLP#7200#AAA,CLP#10800#AAA
0,0.987349,0.957658,0.910992,0.824626,0.679582,0.560789,0.462788,0.381914,0.260095,0.177133,0.146178,0.055950,0.021415,0.003137
1,0.987521,0.958512,0.911507,0.821608,0.672886,0.554045,0.456607,0.376348,0.255680,0.173703,0.143173,0.054467,0.020721,0.002999
2,0.987399,0.957999,0.911080,0.822914,0.675853,0.556498,0.458324,0.377475,0.256047,0.173681,0.143044,0.054206,0.020541,0.002950
3,0.987549,0.957278,0.913511,0.831901,0.689907,0.572151,0.474494,0.393505,0.270639,0.186136,0.154366,0.060555,0.023755,0.003656
4,0.987764,0.959484,0.912896,0.821228,0.669218,0.550894,0.455016,0.376130,0.257114,0.175767,0.145326,0.056154,0.021698,0.003240
5,0.988847,0.959121,0.913728,0.829248,0.683021,0.562584,0.463384,0.381676,0.258943,0.175676,0.144700,0.054858,0.020798,0.002989
6,0.988987,0.958709,0.914102,0.831050,0.686913,0.567778,0.469305,0.387912,0.265026,0.181069,0.149665,0.057744,0.022279,0.003316
7,0.988214,0.958132,0.914597,0.833389,0.691972,0.574553,0.477059,0.396108,0.273085,0.188270,0.156323,0.061693,0.024347,0.003792
8,0.988326,0.958285,0.914893,0.833932,0.692876,0.575681,0.478309,0.397406,0.274339,0.189383,0.157350,0.062302,0.024668,0.003867
9,0.988307,0.958026,0.914258,0.832647,0.690639,0.572851,0.475152,0.394116,0.271148,0.186548,0.154732,0.060749,0.023850,0.003676


In [13]:
bono.get_volatilidad()

,0
CLP#30#AAA,0.000439
CLP#90#AAA,0.000978
CLP#180#AAA,0.001317
CLP#360#AAA,0.002624
CLP#720#AAA,0.007382
CLP#1080#AAA,0.011895
CLP#1440#AAA,0.015563
CLP#1800#AAA,0.018492
CLP#2520#AAA,0.023115
CLP#3240#AAA,0.027187


In [14]:
bono.get_correlacion()

,CLP#30#AAA,CLP#90#AAA,CLP#180#AAA,CLP#360#AAA,CLP#720#AAA,CLP#1080#AAA,CLP#1440#AAA,CLP#1800#AAA,CLP#2520#AAA,CLP#3240#AAA,CLP#3600#AAA,CLP#5400#AAA,CLP#7200#AAA,CLP#10800#AAA
CLP#30#AAA,1.000000,0.697600,0.312874,0.082489,0.082383,0.103570,0.117905,0.128882,0.145612,0.157317,0.161686,0.174381,0.179651,0.183520
CLP#90#AAA,0.697600,1.000000,0.758364,0.019064,-0.238858,-0.261092,-0.263666,-0.262946,-0.258154,-0.250956,-0.247023,-0.228823,-0.215345,-0.198560
CLP#180#AAA,0.312874,0.758364,1.000000,0.510177,0.099438,0.009837,-0.027590,-0.049512,-0.076947,-0.094708,-0.101459,-0.122953,-0.133850,-0.144250
CLP#360#AAA,0.082489,0.019064,0.510177,1.000000,0.864915,0.783715,0.741325,0.714794,0.676459,0.643067,0.627403,0.562317,0.517568,0.463906
CLP#720#AAA,0.082383,-0.238858,0.099438,0.864915,1.000000,0.986872,0.970778,0.956414,0.926330,0.892591,0.875449,0.800086,0.746213,0.680315
CLP#1080#AAA,0.103570,-0.261092,0.009837,0.783715,0.986872,1.000000,0.996361,0.989002,0.966539,0.936876,0.921113,0.849532,0.797191,0.732420
CLP#1440#AAA,0.117905,-0.263666,-0.027590,0.741325,0.970778,0.996361,1.000000,0.997799,0.982935,0.958829,0.945284,0.881071,0.832632,0.771721
CLP#1800#AAA,0.128882,-0.262946,-0.049512,0.714794,0.956414,0.989002,0.997799,1.000000,0.992735,0.974783,0.963788,0.908317,0.864662,0.808603
CLP#2520#AAA,0.145612,-0.258154,-0.076947,0.676459,0.926330,0.966539,0.982935,0.992735,1.000000,0.994496,0.988763,0.951662,0.918253,0.872742
CLP#3240#AAA,0.157317,-0.250956,-0.094708,0.643067,0.892591,0.936876,0.958829,0.974783,0.994496,1.000000,0.998983,0.978590,0.954661,0.919043


In [15]:
bono.get_volatilidad_general()

array([[0.00677462]])

#### Derivados

In [16]:
derivados = cartera.get_derivados()
derivados

In [17]:
derivado = derivados[0]
derivado.get_volatilidad()

,0
CLP#30,0.000325
CLP#90,0.000446
CLP#180,0.000768
CLP#360,0.000964
CLP#720,0.001234
CLP#1080,0.001838
CLP#1440,0.002607
CLP#1800,0.003133
CLP#2520,0.004260
CLP#3240,0.005379


In [18]:
derivado.get_correlacion()

,CLP#30,CLP#90,CLP#180,CLP#360,CLP#720,CLP#1080,CLP#1440,CLP#1800,CLP#2520,CLP#3240,CLP#3600,CLP#5400,CLP#7200,CLP#10800
CLP#30,1.000000,0.714502,0.675601,0.637402,0.719501,0.702052,0.688773,0.664240,0.621252,0.594075,0.602601,0.581762,0.540498,0.537118
CLP#90,0.714502,1.000000,0.851443,0.863799,0.669093,0.690917,0.647384,0.652120,0.635888,0.653034,0.643655,0.559351,0.647765,0.650634
CLP#180,0.675601,0.851443,1.000000,0.947213,0.724598,0.734747,0.720781,0.726089,0.669846,0.649635,0.635094,0.570628,0.650782,0.653226
CLP#360,0.637402,0.863799,0.947213,1.000000,0.686187,0.705772,0.668085,0.666238,0.630025,0.610747,0.589587,0.517932,0.599368,0.601993
CLP#720,0.719501,0.669093,0.724598,0.686187,1.000000,0.991899,0.986685,0.974905,0.947390,0.931029,0.918083,0.903539,0.900610,0.898666
CLP#1080,0.702052,0.690917,0.734747,0.705772,0.991899,1.000000,0.991339,0.978968,0.958856,0.944945,0.927079,0.911679,0.920276,0.918863
CLP#1440,0.688773,0.647384,0.720781,0.668085,0.986685,0.991339,1.000000,0.992639,0.969912,0.950381,0.935904,0.925624,0.931686,0.930104
CLP#1800,0.664240,0.652120,0.726089,0.666238,0.974905,0.978968,0.992639,1.000000,0.975644,0.955407,0.944405,0.925970,0.940865,0.939727
CLP#2520,0.621252,0.635888,0.669846,0.630025,0.947390,0.958856,0.969912,0.975644,1.000000,0.980892,0.973250,0.961875,0.965020,0.963295
CLP#3240,0.594075,0.653034,0.649635,0.610747,0.931029,0.944945,0.950381,0.955407,0.980892,1.000000,0.994324,0.969197,0.976884,0.975367


In [19]:
derivado.get_volatilidad_general()

0.0013413740541173842

Por ejemplo, si deseamos saber la volatilidad de la cartera, se debe invocar al siguiente método:

In [20]:
cartera.set_volatilidad_cartera()
volatilidad = cartera.get_volatilidad_cartera()
volatilidad

0.0019744569418541423

In [21]:
cartera.get_historicos_totales()

,CLP#30#AAA,CLP#90#AAA,CLP#180#AAA,CLP#360#AAA,CLP#720#AAA,CLP#1080#AAA,CLP#1440#AAA,CLP#1800#AAA,CLP#2520#AAA,CLP#3240#AAA,...,CLP#1080,CLP#1440,CLP#1800,CLP#2520,CLP#3240,CLP#3600,CLP#5400,CLP#7200,CLP#10800,Amazon
0,0.987349,0.957658,0.910992,0.824626,0.679582,0.560789,0.462788,0.381914,0.260095,0.177133,...,0.945271,0.920921,0.893143,0.831261,0.766409,0.738088,0.617735,0.525645,0.381113,133.899994
1,0.987521,0.958512,0.911507,0.821608,0.672886,0.554045,0.456607,0.376348,0.255680,0.173703,...,0.945134,0.921105,0.893373,0.831533,0.766046,0.737300,0.617233,0.524469,0.379802,134.690002
2,0.987399,0.957999,0.911080,0.822914,0.675853,0.556498,0.458324,0.377475,0.256047,0.173681,...,0.945224,0.921186,0.893525,0.831692,0.766129,0.737445,0.617343,0.523926,0.379195,132.250000
3,0.987549,0.957278,0.913511,0.831901,0.689907,0.572151,0.474494,0.393505,0.270639,0.186136,...,0.945505,0.921717,0.894395,0.832008,0.767146,0.738167,0.618230,0.525643,0.381076,130.000000
4,0.987764,0.959484,0.912896,0.821228,0.669218,0.550894,0.455016,0.376130,0.257114,0.175767,...,0.945408,0.921966,0.894696,0.833058,0.768410,0.739573,0.620019,0.527619,0.383226,133.520004
5,0.988847,0.959121,0.913728,0.829248,0.683021,0.562584,0.463384,0.381676,0.258943,0.175676,...,0.944585,0.920531,0.892919,0.830680,0.766395,0.737442,0.616786,0.525871,0.381424,130.309998
6,0.988987,0.958709,0.914102,0.831050,0.686913,0.567778,0.469305,0.387912,0.265026,0.181069,...,0.942910,0.917479,0.888932,0.825432,0.760885,0.731264,0.610021,0.516319,0.370984,127.349998
7,0.988214,0.958132,0.914597,0.833389,0.691972,0.574553,0.477059,0.396108,0.273085,0.188270,...,0.941894,0.915942,0.886688,0.822472,0.757257,0.726994,0.605628,0.510754,0.364994,129.110001
8,0.988326,0.958285,0.914893,0.833932,0.692876,0.575681,0.478309,0.397406,0.274339,0.189383,...,0.942167,0.916473,0.887556,0.823905,0.759022,0.728811,0.608453,0.512011,0.366222,127.349998
9,0.988307,0.958026,0.914258,0.832647,0.690639,0.572851,0.475152,0.394116,0.271148,0.186548,...,0.942142,0.916641,0.886975,0.823868,0.760341,0.729935,0.610323,0.513486,0.367769,127.139999


In [22]:
cartera.get_retornos_totales()

,CLP#30#AAA,CLP#90#AAA,CLP#180#AAA,CLP#360#AAA,CLP#720#AAA,CLP#1080#AAA,CLP#1440#AAA,CLP#1800#AAA,CLP#2520#AAA,CLP#3240#AAA,...,CLP#1080,CLP#1440,CLP#1800,CLP#2520,CLP#3240,CLP#3600,CLP#5400,CLP#7200,CLP#10800,Amazon
0,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000174,0.000891,0.000565,-0.003667,-9.902586e-03,-0.012099,-0.013445,-0.014679,-0.017119,-0.019555,...,-1.444339e-04,0.000199,0.000258,0.000327,-0.000474,-0.001068,-0.000813,-0.002239,-0.003447,0.005883
2,-0.000123,-0.000535,-0.000469,0.001589,4.399721e-03,0.004419,0.003754,0.002990,0.001434,-0.000125,...,9.527514e-05,0.000088,0.000169,0.000192,0.000109,0.000196,0.000179,-0.001035,-0.001599,-0.018282
3,0.000151,-0.000753,0.002665,0.010862,2.058162e-02,0.027739,0.034671,0.041589,0.055424,0.069258,...,2.964891e-04,0.000576,0.000974,0.000379,0.001326,0.000979,0.001436,0.003271,0.004949,-0.017160
4,0.000217,0.002302,-0.000674,-0.012913,-3.044799e-02,-0.037861,-0.041915,-0.045160,-0.051267,-0.057319,...,-1.026542e-04,0.000270,0.000336,0.001262,0.001646,0.001903,0.002888,0.003751,0.005625,0.026717
5,0.001096,-0.000379,0.000911,0.009718,2.041612e-02,0.020998,0.018222,0.014639,0.007088,-0.000517,...,-8.700799e-04,-0.001557,-0.001988,-0.002859,-0.002626,-0.002885,-0.005228,-0.003318,-0.004712,-0.024335
6,0.000141,-0.000429,0.000409,0.002171,5.681811e-03,0.009190,0.012698,0.016205,0.023220,0.030234,...,-1.775740e-03,-0.003322,-0.004474,-0.006337,-0.007215,-0.008413,-0.011028,-0.018331,-0.027753,-0.022977
7,-0.000782,-0.000602,0.000542,0.002811,7.338297e-03,0.011863,0.016386,0.020909,0.029956,0.039002,...,-1.077137e-03,-0.001676,-0.002527,-0.003593,-0.004780,-0.005856,-0.007227,-0.010836,-0.016280,0.013726
8,0.000113,0.000159,0.000323,0.000651,1.305844e-03,0.001961,0.002616,0.003272,0.004582,0.005892,...,2.889848e-04,0.000579,0.000978,0.001741,0.002328,0.002496,0.004654,0.002458,0.003359,-0.013726
9,-0.000019,-0.000270,-0.000695,-0.001542,-3.234773e-03,-0.004928,-0.006621,-0.008313,-0.011699,-0.015085,...,-2.634027e-05,0.000183,-0.000655,-0.000045,0.001737,0.001541,0.003068,0.002876,0.004216,-0.001650


In [23]:
cartera.get_volatilidades_totales()

,0
CLP#30#AAA,0.000439
CLP#90#AAA,0.000978
CLP#180#AAA,0.001317
CLP#360#AAA,0.002624
CLP#720#AAA,0.007382
CLP#1080#AAA,0.011895
CLP#1440#AAA,0.015563
CLP#1800#AAA,0.018492
CLP#2520#AAA,0.023115
CLP#3240#AAA,0.027187


In [24]:
cartera.get_correlacion()

,CLP#30#AAA,CLP#90#AAA,CLP#180#AAA,CLP#360#AAA,CLP#720#AAA,CLP#1080#AAA,CLP#1440#AAA,CLP#1800#AAA,CLP#2520#AAA,CLP#3240#AAA,...,CLP#1080,CLP#1440,CLP#1800,CLP#2520,CLP#3240,CLP#3600,CLP#5400,CLP#7200,CLP#10800,Amazon
CLP#30#AAA,1.000000,0.697600,0.312874,0.082489,0.082383,0.103570,0.117905,0.128882,0.145612,0.157317,...,0.002350,0.016606,0.021647,0.029627,-0.015883,-0.011518,-0.006220,-0.018737,-0.019066,-0.085660
CLP#90#AAA,0.697600,1.000000,0.758364,0.019064,-0.238858,-0.261092,-0.263666,-0.262946,-0.258154,-0.250956,...,0.132254,0.147718,0.162503,0.169474,0.128821,0.130959,0.116081,0.152477,0.154023,-0.054682
CLP#180#AAA,0.312874,0.758364,1.000000,0.510177,0.099438,0.009837,-0.027590,-0.049512,-0.076947,-0.094708,...,0.166101,0.179095,0.194234,0.187548,0.200966,0.213921,0.166663,0.214538,0.216360,-0.056893
CLP#360#AAA,0.082489,0.019064,0.510177,1.000000,0.864915,0.783715,0.741325,0.714794,0.676459,0.643067,...,0.096556,0.104732,0.113935,0.108929,0.139061,0.150702,0.112339,0.108835,0.108432,-0.061551
CLP#720#AAA,0.082383,-0.238858,0.099438,0.864915,1.000000,0.986872,0.970778,0.956414,0.926330,0.892591,...,0.042692,0.047292,0.049942,0.051178,0.051420,0.053097,0.041722,0.013413,0.012086,-0.072990
CLP#1080#AAA,0.103570,-0.261092,0.009837,0.783715,0.986872,1.000000,0.996361,0.989002,0.966539,0.936876,...,0.030100,0.034246,0.034496,0.036986,0.028455,0.027264,0.025555,-0.009658,-0.011248,-0.070285
CLP#1440#AAA,0.117905,-0.263666,-0.027590,0.741325,0.970778,0.996361,1.000000,0.997799,0.982935,0.958829,...,0.025201,0.029537,0.028406,0.030675,0.018098,0.015478,0.019211,-0.020239,-0.022002,-0.061554
CLP#1800#AAA,0.128882,-0.262946,-0.049512,0.714794,0.956414,0.989002,0.997799,1.000000,0.992735,0.974783,...,0.022628,0.027360,0.025212,0.026635,0.011422,0.007789,0.015491,-0.027240,-0.029141,-0.050695
CLP#2520#AAA,0.145612,-0.258154,-0.076947,0.676459,0.926330,0.966539,0.982935,0.992735,1.000000,0.994496,...,0.019684,0.025332,0.021615,0.020754,0.001858,-0.003320,0.010492,-0.037300,-0.039419,-0.027641
CLP#3240#AAA,0.157317,-0.250956,-0.094708,0.643067,0.892591,0.936876,0.958829,0.974783,0.994496,1.000000,...,0.017712,0.024163,0.019263,0.016189,-0.005242,-0.011571,0.006845,-0.044485,-0.046758,-0.006746


In [25]:
cartera.get_covarianza()

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,1.923124e-07,2.992485e-07,1.806471e-07,9.492697e-08,2.667039e-07,5.402788e-07,8.047011e-07,1.045162e-06,1.476037e-06,1.875610e-06,...,1.894174e-09,1.898580e-08,2.974223e-08,5.535117e-08,-3.746525e-08,-3.020811e-08,-2.461242e-08,-9.599660e-08,-1.465441e-07,-6.653242e-07
1,2.992485e-07,9.568496e-07,9.766923e-07,4.893629e-08,-1.724844e-06,-3.038058e-06,-4.013980e-06,-4.756384e-06,-5.837081e-06,-6.673947e-06,...,2.377865e-07,3.767210e-07,4.980327e-07,7.062527e-07,6.778040e-07,7.661511e-07,1.024549e-06,1.742488e-06,2.640622e-06,-9.473536e-07
2,1.806471e-07,9.766923e-07,1.733468e-06,1.762666e-06,9.664898e-07,1.540638e-07,-5.653389e-07,-1.205477e-06,-2.341761e-06,-3.390050e-06,...,4.019641e-07,6.147583e-07,8.012321e-07,1.051976e-06,1.423231e-06,1.684489e-06,1.979914e-06,3.299951e-06,4.992662e-06,-1.326689e-06
3,9.492697e-08,4.893629e-08,1.762666e-06,6.886239e-06,1.675529e-05,2.446409e-05,3.027597e-05,3.468646e-05,4.103247e-05,4.587857e-05,...,4.657239e-07,7.165322e-07,9.367478e-07,1.217786e-06,1.962869e-06,2.365197e-06,2.659923e-06,3.336597e-06,4.987100e-06,-2.860733e-06
4,2.667039e-07,-1.724844e-06,9.664898e-07,1.675529e-05,5.449727e-05,8.666187e-05,1.115336e-04,1.305633e-04,1.580696e-04,1.791442e-04,...,5.792799e-07,9.102105e-07,1.155120e-06,1.609566e-06,2.041807e-06,2.344289e-06,2.779100e-06,1.156814e-06,1.563788e-06,-9.543332e-06
5,5.402788e-07,-3.038058e-06,1.540638e-07,2.446409e-05,8.666187e-05,1.415011e-04,1.844570e-04,2.175529e-04,2.657629e-04,3.029878e-04,...,6.581157e-07,1.062059e-06,1.285655e-06,1.874383e-06,1.820674e-06,1.939654e-06,2.742910e-06,-1.342207e-06,-2.345117e-06,-1.480783e-05
6,8.047011e-07,-4.013980e-06,-5.653389e-07,3.027597e-05,1.115336e-04,1.844570e-04,2.422127e-04,2.871636e-04,3.536049e-04,4.056977e-04,...,7.208845e-07,1.198477e-06,1.385092e-06,2.033820e-06,1.515029e-06,1.440667e-06,2.697682e-06,-3.679912e-06,-6.001519e-06,-1.696687e-05
7,1.045162e-06,-4.756384e-06,-1.205477e-06,3.468646e-05,1.305633e-04,2.175529e-04,2.871636e-04,3.419602e-04,4.243420e-04,4.900705e-04,...,7.691080e-07,1.319073e-06,1.460743e-06,2.098327e-06,1.136075e-06,8.614813e-07,2.584806e-06,-5.884847e-06,-9.444759e-06,-1.660348e-05
8,1.476037e-06,-5.837081e-06,-2.341761e-06,4.103247e-05,1.580696e-04,2.657629e-04,3.536049e-04,4.243420e-04,5.343062e-04,6.249726e-04,...,8.362949e-07,1.526587e-06,1.565392e-06,2.043750e-06,2.310400e-07,-4.589070e-07,2.188184e-06,-1.007284e-05,-1.596994e-05,-1.131621e-05
9,1.875610e-06,-6.673947e-06,-3.390050e-06,4.587857e-05,1.791442e-04,3.029878e-04,4.056977e-04,4.900705e-04,6.249726e-04,7.391382e-04,...,8.851077e-07,1.712650e-06,1.640825e-06,1.875073e-06,-7.665343e-07,-1.881465e-06,1.679156e-06,-1.412939e-05,-2.228013e-05,-3.248410e-06
